## BIG DATA MANAGEMENT - FINAL PROJECT - PART/1
BY MERCAN KARACABEY

In [2]:
sudo service postgresql start

Starting PostgreSQL 11 database server: main.


In [4]:
psql -U postgres -c "\l"

                                        List of databases
              Name              |  Owner   | Encoding | Collate |  Ctype  |   Access privileges   
--------------------------------+----------+----------+---------+---------+-----------------------
 barcelona_immigrants_emigrants | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 imdb2                          | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 insurance_sample               | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 postgres                       | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 template0                      | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
                                |          |          |         |         | postgres=CTc/postgres
 template1                      | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
                                |          |          |         |         | postgres=CTc/postgres
(6 rows)



### Environment Variables Config

In [5]:
datadir=~/data
imdbdir=$datadir/imdb

In [6]:
ls -l $datadir

total 32
drwxr-xr-x 3 jovyan jovyan  4096 Dec 15 10:57 comtrade_s1
drwxr-xr-x 2 jovyan jovyan 24576 Dec 15 10:57 he_sisli
drwxr-xr-x 4 jovyan jovyan  4096 Dec 15 10:57 imdb


In [7]:
mkdir -p $imdbdir/tsv2

In [8]:
find $imdbdir/tsv -mindepth 1 | \
    parallel -k -j0 "basenm=\$(basename {});
        if [ ! -e ${imdbdir}/tsv2/\${basenm%.gz} ];
        then
            gunzip -c {} > \
            ${imdbdir}/tsv2/\${basenm%.gz};
        fi
        "

In [9]:
mkdir -p $imdbdir/tsv3

In [12]:
mkdir -p $imdbdir/csv

In [ ]:
### Create tablespace & database & tables

In [13]:
psql -U postgres -c "\l"

                                        List of databases
              Name              |  Owner   | Encoding | Collate |  Ctype  |   Access privileges   
--------------------------------+----------+----------+---------+---------+-----------------------
 barcelona_immigrants_emigrants | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 imdb2                          | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 insurance_sample               | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 postgres                       | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 template0                      | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
                                |          |          |         |         | postgres=CTc/postgres
 template1                      | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
                                |          |          |         |         | postgres=CTc/postgres
(6 rows)



In [14]:
sudo mkdir -p /pg1

In [15]:
stat -c "%U %G" /pg1

root root


In [16]:
sudo chown -R postgres:postgres /pg1

In [17]:
stat -c "%U %G" /pg1

postgres postgres


In [19]:
%load_ext sql

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [20]:
%sql postgres://postgres@localhost/postgres

'Connected: postgres@postgres'

In [21]:
%%sql
-- create a new tablespace
END;
CREATE TABLESPACE pg1 LOCATION '/pg1';

 * postgres://postgres@localhost/postgres
Done.
Done.


[]

In [22]:
%%sql
-- create a new database imdb1 on pg1 tablespace
END;
CREATE DATABASE imdb1
TABLESPACE = pg1;

 * postgres://postgres@localhost/postgres
Done.
Done.


[]

In [23]:
%sql postgres://postgres@localhost/imdb1

'Connected: postgres@imdb1'

In [24]:
%sql postgres://postgres@localhost/imdb1

'Connected: postgres@imdb1'

In [25]:
%%sql
-- create a new table
CREATE TABLE title_ratings
(
  tconst text,
  averageRating numeric,
  numVotes integer
);

 * postgres://postgres@localhost/imdb1
   postgres://postgres@localhost/postgres
Done.


[]

In [27]:
psql -U postgres -d imdb1 -c "\copy title_ratings from $imdbdir/tsv3/title.ratings2.tsv"

COPY 767041


In [29]:
psql -U postgres -d imdb1 -c "\dt+"

                        List of relations
 Schema |     Name      | Type  |  Owner   | Size  | Description 
--------+---------------+-------+----------+-------+-------------
 public | title_ratings | table | postgres | 38 MB | 
(1 row)



### SELECT QUERIES

In [30]:
%%sql
SELECT *
FROM title_ratings
LIMIT 5;

 * postgres://postgres@localhost/imdb1
   postgres://postgres@localhost/postgres
5 rows affected.


tconst,averagerating,numvotes
tt0000001,5.8,1306
tt0000002,6.5,149
tt0000003,6.6,885
tt0000004,6.4,92
tt0000005,6.2,1565


In [31]:
%%sql
ALTER TABLE title_ratings
ADD CONSTRAINT title_ratings_pk 
PRIMARY KEY (tconst);

 * postgres://postgres@localhost/imdb1
   postgres://postgres@localhost/postgres
Done.


[]

In [32]:
%%sql
SELECT * FROM crew_molten
LIMIT 5;

 * postgres://postgres@localhost/imdb1
   postgres://postgres@localhost/postgres
(psycopg2.ProgrammingError) relation "crew_molten" does not exist
LINE 1: SELECT * FROM crew_molten
                      ^
 [SQL: 'SELECT * FROM crew_molten\nLIMIT 5;'] (Background on this error at: http://sqlalche.me/e/f405)


In [36]:
psql -U postgres -c "\c imdb2"

You are now connected to database "imdb2" as user "postgres".


In [38]:
psql -U postgres -d imdb2 -c "\dt+"

                             List of relations
 Schema |         Name          | Type  |  Owner   |  Size   | Description 
--------+-----------------------+-------+----------+---------+-------------
 public | crew_molten           | table | postgres | 679 MB  | 
 public | genres_molten         | table | postgres | 334 MB  | 
 public | name_basics           | table | postgres | 676 MB  | 
 public | title_basics          | table | postgres | 516 MB  | 
 public | title_crew            | table | postgres | 260 MB  | 
 public | title_episode         | table | postgres | 167 MB  | 
 public | title_principals_melt | table | postgres | 1262 MB | 
 public | title_ratings         | table | postgres | 38 MB   | 
(8 rows)



In [39]:
%load_ext sql
%sql postgres://postgres@localhost/imdb2

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: postgres@imdb2'

In [40]:
%%sql
SELECT * FROM crew_molten
LIMIT 5;

   postgres://postgres@localhost/imdb1
 * postgres://postgres@localhost/imdb2
   postgres://postgres@localhost/postgres
5 rows affected.


tconst,profession,nconst
tt0000001,director,nm0005690
tt0000001,writer,\N
tt0000002,director,nm0721526
tt0000002,writer,\N
tt0000003,director,nm0721526


### SELECT QUERIES WITH JOIN

In [41]:
%%sql

SELECT tb.tconst, tb.originaltitle, tb.startyear,
    tb.runtimeminutes, tb.genres, tp.principalcast
    FROM title_basics tb
    LEFT JOIN title_principals_melt tp ON tb.tconst=tp.tconst
  WHERE tb.originaltitle ~ 'Godfather.*Part'
	AND tb.genres ~ '(?i)drama'
	AND NOT tb.genres ~ '(?i)comedy'
	AND tb.startyear <= 1990
  ORDER BY tb.runtimeminutes;

   postgres://postgres@localhost/imdb1
 * postgres://postgres@localhost/imdb2
   postgres://postgres@localhost/postgres
20 rows affected.


tconst,originaltitle,startyear,runtimeminutes,genres,principalcast
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0000199
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0000338
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0000412
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0000473
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0001735
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0178874
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0296636
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0539794
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0701374
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0932336


In [42]:
%%sql

SELECT tb.tconst, tb.originaltitle, tb.startyear,
    tb.runtimeminutes, tb.genres, tp.principalcast,tc.directors,tc.writers
    FROM title_basics tb
    LEFT JOIN title_principals_melt tp ON tb.tconst=tp.tconst
    INNER JOIN title_crew tc ON tb.tconst=tc.tconst
  WHERE tb.originaltitle ~ 'Godfather.*Part'
	AND tb.genres ~ '(?i)drama'
	AND NOT tb.genres ~ '(?i)comedy'
	AND tb.startyear <= 1990
  ORDER BY tb.runtimeminutes;

   postgres://postgres@localhost/imdb1
 * postgres://postgres@localhost/imdb2
   postgres://postgres@localhost/postgres
20 rows affected.


tconst,originaltitle,startyear,runtimeminutes,genres,principalcast,directors,writers
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0000199,nm0000338,"nm0701374,nm0000338"
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0000338,nm0000338,"nm0701374,nm0000338"
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0000412,nm0000338,"nm0701374,nm0000338"
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0000473,nm0000338,"nm0701374,nm0000338"
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0001735,nm0000338,"nm0701374,nm0000338"
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0178874,nm0000338,"nm0701374,nm0000338"
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0296636,nm0000338,"nm0701374,nm0000338"
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0539794,nm0000338,"nm0701374,nm0000338"
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0701374,nm0000338,"nm0701374,nm0000338"
tt0099674,The Godfather: Part III,1990,162,"Crime,Drama",nm0932336,nm0000338,"nm0701374,nm0000338"


In [3]:
%%sql

SELECT subq1.*,tp.principalcast,tc.directors, 
    FROM (SELECT * FROM title_basics 
  WHERE 
    tb.genres ~ '(?i)crime'
	AND tb.startyear <= 1950
  ORDER BY tb.runtimeminutes) subq1
 
LEFT JOIN title_principals_melt tp ON subq1.tconst=tp.tconst
INNER JOIN title_crew tc ON subq1.tconst=tc.tconst
LIMIT 20;

java.lang.RuntimeException:  No datasource

java.lang.RuntimeException:  No datasource